In [1]:
#filter eclipse all
import pandas as pd

csmr12_eclipse_initial = pd.read_csv('csmr12/csmr12_eclipse_initial.csv')
csmr12_eclipse_last = pd.read_csv('csmr12/csmr12_eclipse_last.csv')
csmr12_eclipse_label = pd.read_csv('csmr12/csmr12_eclipse_label.csv')
csmr12_eclipse_userid = pd.read_csv('bugs_assignee_init_final.txt', names=['bug_id', 'init', 'final'], sep=';')

In [2]:
#issameproduct = pd.merge(csmr12_eclipse_initial[['bug_id','product_id']],csmr12_eclipse_last[['bug_id','product_id']], on='bug_id')
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_label = pd.merge(csmr12_eclipse_label, issameproduct[['bug_id']], on='bug_id')

In [3]:
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_userid[['bug_id', 'init']], on='bug_id')
csmr12_eclipse_initial['assigned_to'] = csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['creation_ts']
#del csmr12_eclipse_initial['index']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_label, on='bug_id')
threshold_interval = csmr12_eclipse_initial[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, threshold_interval, on='product_id')
csmr12_eclipse_initial = csmr12_eclipse_initial[csmr12_eclipse_initial.interval>csmr12_eclipse_initial.threshold]

median_interval = csmr12_eclipse_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, median_interval, on='product_id')
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval<=csmr12_eclipse_initial.med, 'label'] = 'fast'
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval>csmr12_eclipse_initial.med, 'label'] = 'slow'

In [4]:
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_userid[['bug_id', 'final']], on='bug_id')
csmr12_eclipse_last['assigned_to'] = csmr12_eclipse_last['final']
del csmr12_eclipse_last['final']
del csmr12_eclipse_last['creation_ts']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_label, on='bug_id')
threshold_interval = csmr12_eclipse_last[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, threshold_interval, on='product_id')
csmr12_eclipse_last = csmr12_eclipse_last[csmr12_eclipse_last.interval>csmr12_eclipse_last.threshold]

median_interval = csmr12_eclipse_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, median_interval, on='product_id')
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval<=csmr12_eclipse_last.med, 'label'] = 'fast'
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval>csmr12_eclipse_last.med, 'label'] = 'slow'

In [5]:
del csmr12_eclipse_last['interval']
del csmr12_eclipse_last['threshold']
del csmr12_eclipse_last['med']


csmr12_eclipse_severity = csmr12_eclipse_last.copy()
del csmr12_eclipse_severity['bug_severity']
csmr12_eclipse_severity['bug_severity'] = csmr12_eclipse_initial['bug_severity']

csmr12_eclipse_priority = csmr12_eclipse_last.copy()
del csmr12_eclipse_priority['priority']
csmr12_eclipse_priority['priority'] = csmr12_eclipse_initial['priority']

csmr12_eclipse_platform = csmr12_eclipse_last.copy()
del csmr12_eclipse_platform['rep_platform']
csmr12_eclipse_platform['rep_platform'] = csmr12_eclipse_initial['rep_platform']

csmr12_eclipse_os = csmr12_eclipse_last.copy()
del csmr12_eclipse_os['op_sys']
csmr12_eclipse_os['op_sys'] = csmr12_eclipse_initial['op_sys']

csmr12_eclipse_assignee = csmr12_eclipse_last.copy()
del csmr12_eclipse_assignee['assigned_to']
csmr12_eclipse_assignee['assigned_to'] = csmr12_eclipse_initial['assigned_to']

csmr12_eclipse_component = csmr12_eclipse_last.copy()
del csmr12_eclipse_component['component_id']
csmr12_eclipse_component['component_id'] = csmr12_eclipse_initial['component_id']

In [6]:
csmr12_eclipse_severity[csmr12_eclipse_severity.product_id==1].to_csv('csmr12attr/csmr12_eclipse_severity_Platform.csv',index=False)
csmr12_eclipse_severity[csmr12_eclipse_severity.product_id==2].to_csv('csmr12attr/csmr12_eclipse_severity_PDE.csv',index=False)
csmr12_eclipse_severity[csmr12_eclipse_severity.product_id==3].to_csv('csmr12attr/csmr12_eclipse_severity_JDT.csv',index=False)
csmr12_eclipse_severity[csmr12_eclipse_severity.product_id==4].to_csv('csmr12attr/csmr12_eclipse_severity_CDT.csv',index=False)
csmr12_eclipse_severity[csmr12_eclipse_severity.product_id==5].to_csv('csmr12attr/csmr12_eclipse_severity_GEF.csv',index=False)

csmr12_eclipse_priority[csmr12_eclipse_priority.product_id==1].to_csv('csmr12attr/csmr12_eclipse_priority_Platform.csv',index=False)
csmr12_eclipse_priority[csmr12_eclipse_priority.product_id==2].to_csv('csmr12attr/csmr12_eclipse_priority_PDE.csv',index=False)
csmr12_eclipse_priority[csmr12_eclipse_priority.product_id==3].to_csv('csmr12attr/csmr12_eclipse_priority_JDT.csv',index=False)
csmr12_eclipse_priority[csmr12_eclipse_priority.product_id==4].to_csv('csmr12attr/csmr12_eclipse_priority_CDT.csv',index=False)
csmr12_eclipse_priority[csmr12_eclipse_priority.product_id==5].to_csv('csmr12attr/csmr12_eclipse_priority_GEF.csv',index=False)

csmr12_eclipse_platform[csmr12_eclipse_platform.product_id==1].to_csv('csmr12attr/csmr12_eclipse_platform_Platform.csv',index=False)
csmr12_eclipse_platform[csmr12_eclipse_platform.product_id==2].to_csv('csmr12attr/csmr12_eclipse_platform_PDE.csv',index=False)
csmr12_eclipse_platform[csmr12_eclipse_platform.product_id==3].to_csv('csmr12attr/csmr12_eclipse_platform_JDT.csv',index=False)
csmr12_eclipse_platform[csmr12_eclipse_platform.product_id==4].to_csv('csmr12attr/csmr12_eclipse_platform_CDT.csv',index=False)
csmr12_eclipse_platform[csmr12_eclipse_platform.product_id==5].to_csv('csmr12attr/csmr12_eclipse_platform_GEF.csv',index=False)

csmr12_eclipse_os[csmr12_eclipse_os.product_id==1].to_csv('csmr12attr/csmr12_eclipse_os_Platform.csv',index=False)
csmr12_eclipse_os[csmr12_eclipse_os.product_id==2].to_csv('csmr12attr/csmr12_eclipse_os_PDE.csv',index=False)
csmr12_eclipse_os[csmr12_eclipse_os.product_id==3].to_csv('csmr12attr/csmr12_eclipse_os_JDT.csv',index=False)
csmr12_eclipse_os[csmr12_eclipse_os.product_id==4].to_csv('csmr12attr/csmr12_eclipse_os_CDT.csv',index=False)
csmr12_eclipse_os[csmr12_eclipse_os.product_id==5].to_csv('csmr12attr/csmr12_eclipse_os_GEF.csv',index=False)

csmr12_eclipse_assignee[csmr12_eclipse_assignee.product_id==1].to_csv('csmr12attr/csmr12_eclipse_assignee_Platform.csv',index=False)
csmr12_eclipse_assignee[csmr12_eclipse_assignee.product_id==2].to_csv('csmr12attr/csmr12_eclipse_assignee_PDE.csv',index=False)
csmr12_eclipse_assignee[csmr12_eclipse_assignee.product_id==3].to_csv('csmr12attr/csmr12_eclipse_assignee_JDT.csv',index=False)
csmr12_eclipse_assignee[csmr12_eclipse_assignee.product_id==4].to_csv('csmr12attr/csmr12_eclipse_assignee_CDT.csv',index=False)
csmr12_eclipse_assignee[csmr12_eclipse_assignee.product_id==5].to_csv('csmr12attr/csmr12_eclipse_assignee_GEF.csv',index=False)

csmr12_eclipse_component[csmr12_eclipse_component.product_id==1].to_csv('csmr12attr/csmr12_eclipse_component_Platform.csv',index=False)
csmr12_eclipse_component[csmr12_eclipse_component.product_id==2].to_csv('csmr12attr/csmr12_eclipse_component_PDE.csv',index=False)
csmr12_eclipse_component[csmr12_eclipse_component.product_id==3].to_csv('csmr12attr/csmr12_eclipse_component_JDT.csv',index=False)
csmr12_eclipse_component[csmr12_eclipse_component.product_id==4].to_csv('csmr12attr/csmr12_eclipse_component_CDT.csv',index=False)
csmr12_eclipse_component[csmr12_eclipse_component.product_id==5].to_csv('csmr12attr/csmr12_eclipse_component_GEF.csv',index=False)

In [7]:
#filter mozilla all
import pandas as pd

csmr12_mozilla_initial = pd.read_csv('csmr12/csmr12_mozilla_initial.csv')
csmr12_mozilla_last = pd.read_csv('csmr12/csmr12_mozilla_last.csv')
csmr12_mozilla_label = pd.read_csv('csmr12/csmr12_mozilla_label.csv')


#issameproduct = pd.merge(csmr12_mozilla_initial[['bug_id','product_id']],csmr12_mozilla_last[['bug_id','product_id']], on='bug_id')
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_label = pd.merge(csmr12_mozilla_label, issameproduct[['bug_id']], on='bug_id')


del csmr12_mozilla_initial['creation_ts']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, csmr12_mozilla_label, on='bug_id')
threshold_interval = csmr12_mozilla_initial[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, threshold_interval, on='product_id')
csmr12_mozilla_initial = csmr12_mozilla_initial[csmr12_mozilla_initial.interval>csmr12_mozilla_initial.threshold]

median_interval = csmr12_mozilla_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, median_interval, on='product_id')
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval<=csmr12_mozilla_initial.med, 'label'] = 'fast'
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval>csmr12_mozilla_initial.med, 'label'] = 'slow'


del csmr12_mozilla_last['creation_ts']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, csmr12_mozilla_label, on='bug_id')
threshold_interval = csmr12_mozilla_last[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, threshold_interval, on='product_id')
csmr12_mozilla_last = csmr12_mozilla_last[csmr12_mozilla_last.interval>csmr12_mozilla_last.threshold]

median_interval = csmr12_mozilla_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, median_interval, on='product_id')
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval<=csmr12_mozilla_last.med, 'label'] = 'fast'
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval>csmr12_mozilla_last.med, 'label'] = 'slow'

In [8]:
del csmr12_mozilla_last['interval']
del csmr12_mozilla_last['threshold']
del csmr12_mozilla_last['med']

csmr12_mozilla_severity = csmr12_mozilla_last.copy()
del csmr12_mozilla_severity['bug_severity']
csmr12_mozilla_severity['bug_severity'] = csmr12_mozilla_initial['bug_severity']

csmr12_mozilla_priority = csmr12_mozilla_last.copy()
del csmr12_mozilla_priority['priority']
csmr12_mozilla_priority['priority'] = csmr12_mozilla_initial['priority']

csmr12_mozilla_platform = csmr12_mozilla_last.copy()
del csmr12_mozilla_platform['rep_platform']
csmr12_mozilla_platform['rep_platform'] = csmr12_mozilla_initial['rep_platform']

csmr12_mozilla_os = csmr12_mozilla_last.copy()
del csmr12_mozilla_os['op_sys']
csmr12_mozilla_os['op_sys'] = csmr12_mozilla_initial['op_sys']

csmr12_mozilla_assignee = csmr12_mozilla_last.copy()
del csmr12_mozilla_assignee['assigned_to']
csmr12_mozilla_assignee['assigned_to'] = csmr12_mozilla_initial['assigned_to']

csmr12_mozilla_component = csmr12_mozilla_last.copy()
del csmr12_mozilla_component['component_id']
csmr12_mozilla_component['component_id'] = csmr12_mozilla_initial['component_id']

#csmr12_mozilla_severity.to_csv('csmr12-2-2/csmr12_mozilla_severity.csv',index=False)
#csmr12_mozilla_priority.to_csv('csmr12-2-2/csmr12_mozilla_priority.csv',index=False)
#csmr12_mozilla_platform.to_csv('csmr12-2-2/csmr12_mozilla_platform.csv',index=False)
#csmr12_mozilla_os.to_csv('csmr12-2-2/csmr12_mozilla_os.csv',index=False)
#csmr12_mozilla_assignee.to_csv('csmr12-2-2/csmr12_mozilla_assignee.csv',index=False)
#csmr12_mozilla_component.to_csv('csmr12-2-2/csmr12_mozilla_component.csv',index=False)
#csmr12_mozilla_last.to_csv('csmr12-2-2/csmr12_mozilla.csv',index=False)

In [9]:
csmr12_mozilla_severity[csmr12_mozilla_severity.product_id==1].to_csv('csmr12attr/csmr12_mozilla_severity_Core.csv',index=False)
csmr12_mozilla_severity[csmr12_mozilla_severity.product_id==19].to_csv('csmr12attr/csmr12_mozilla_severity_Bugzilla.csv',index=False)
csmr12_mozilla_severity[csmr12_mozilla_severity.product_id==21].to_csv('csmr12attr/csmr12_mozilla_severity_Firefox.csv',index=False)
csmr12_mozilla_severity[csmr12_mozilla_severity.product_id==23].to_csv('csmr12attr/csmr12_mozilla_severity_Thunderbird.csv',index=False)
csmr12_mozilla_severity[csmr12_mozilla_severity.product_id==25].to_csv('csmr12attr/csmr12_mozilla_severity_Seamonkey.csv',index=False)

csmr12_mozilla_priority[csmr12_mozilla_priority.product_id==1].to_csv('csmr12attr/csmr12_mozilla_priority_Core.csv',index=False)
csmr12_mozilla_priority[csmr12_mozilla_priority.product_id==19].to_csv('csmr12attr/csmr12_mozilla_priority_Bugzilla.csv',index=False)
csmr12_mozilla_priority[csmr12_mozilla_priority.product_id==21].to_csv('csmr12attr/csmr12_mozilla_priority_Firefox.csv',index=False)
csmr12_mozilla_priority[csmr12_mozilla_priority.product_id==23].to_csv('csmr12attr/csmr12_mozilla_priority_Thunderbird.csv',index=False)
csmr12_mozilla_priority[csmr12_mozilla_priority.product_id==25].to_csv('csmr12attr/csmr12_mozilla_priority_Seamonkey.csv',index=False)

csmr12_mozilla_platform[csmr12_mozilla_platform.product_id==1].to_csv('csmr12attr/csmr12_mozilla_platform_Core.csv',index=False)
csmr12_mozilla_platform[csmr12_mozilla_platform.product_id==19].to_csv('csmr12attr/csmr12_mozilla_platform_Bugzilla.csv',index=False)
csmr12_mozilla_platform[csmr12_mozilla_platform.product_id==21].to_csv('csmr12attr/csmr12_mozilla_platform_Firefox.csv',index=False)
csmr12_mozilla_platform[csmr12_mozilla_platform.product_id==23].to_csv('csmr12attr/csmr12_mozilla_platform_Thunderbird.csv',index=False)
csmr12_mozilla_platform[csmr12_mozilla_platform.product_id==25].to_csv('csmr12attr/csmr12_mozilla_platform_Seamonkey.csv',index=False)

csmr12_mozilla_os[csmr12_mozilla_os.product_id==1].to_csv('csmr12attr/csmr12_mozilla_os_Core.csv',index=False)
csmr12_mozilla_os[csmr12_mozilla_os.product_id==19].to_csv('csmr12attr/csmr12_mozilla_os_Bugzilla.csv',index=False)
csmr12_mozilla_os[csmr12_mozilla_os.product_id==21].to_csv('csmr12attr/csmr12_mozilla_os_Firefox.csv',index=False)
csmr12_mozilla_os[csmr12_mozilla_os.product_id==23].to_csv('csmr12attr/csmr12_mozilla_os_Thunderbird.csv',index=False)
csmr12_mozilla_os[csmr12_mozilla_os.product_id==25].to_csv('csmr12attr/csmr12_mozilla_os_Seamonkey.csv',index=False)

csmr12_mozilla_assignee[csmr12_mozilla_assignee.product_id==1].to_csv('csmr12attr/csmr12_mozilla_assignee_Core.csv',index=False)
csmr12_mozilla_assignee[csmr12_mozilla_assignee.product_id==19].to_csv('csmr12attr/csmr12_mozilla_assignee_Bugzilla.csv',index=False)
csmr12_mozilla_assignee[csmr12_mozilla_assignee.product_id==21].to_csv('csmr12attr/csmr12_mozilla_assignee_Firefox.csv',index=False)
csmr12_mozilla_assignee[csmr12_mozilla_assignee.product_id==23].to_csv('csmr12attr/csmr12_mozilla_assignee_Thunderbird.csv',index=False)
csmr12_mozilla_assignee[csmr12_mozilla_assignee.product_id==25].to_csv('csmr12attr/csmr12_mozilla_assignee_Seamonkey.csv',index=False)

csmr12_mozilla_component[csmr12_mozilla_component.product_id==1].to_csv('csmr12attr/csmr12_mozilla_component_Core.csv',index=False)
csmr12_mozilla_component[csmr12_mozilla_component.product_id==19].to_csv('csmr12attr/csmr12_mozilla_component_Bugzilla.csv',index=False)
csmr12_mozilla_component[csmr12_mozilla_component.product_id==21].to_csv('csmr12attr/csmr12_mozilla_component_Firefox.csv',index=False)
csmr12_mozilla_component[csmr12_mozilla_component.product_id==23].to_csv('csmr12attr/csmr12_mozilla_component_Thunderbird.csv',index=False)
csmr12_mozilla_component[csmr12_mozilla_component.product_id==25].to_csv('csmr12attr/csmr12_mozilla_component_Seamonkey.csv',index=False)

In [10]:
import os
import pandas as pd

for filename in os.listdir(r'csmr12attr'):
    if filename.endswith('.csv'):
        tmp = pd.read_csv('csmr12attr/'+filename)
        del tmp['product_id']
        del tmp['bug_id']
        tmp['tag'] = tmp['label']
        del tmp['label']
        tmp.to_csv('csmr12attr/'+filename,index=False)
